<a href="https://colab.research.google.com/github/upashanadutta23/DLPROJECT/blob/main/sp25_UpaHarYam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00


In [ ]:
import re
import torch
from datasets import load_dataset, load_from_disk, DatasetDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

In [ ]:
raw_dataset = load_dataset("rajpurkar/squad_v2")
print(raw_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [ ]:
split_dataset = raw_dataset["train"].train_test_split(
    test_size  = 10000,
    shuffle = True,
    seed = 42,
)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 120319
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10000
    })
})


In [ ]:
train_dataset = split_dataset["train"]
validation_dataset = split_dataset["test"]
test_dataset = raw_dataset["validation"]
print(f"Train Dataset is {train_dataset}\n")
print(f"Validation Dataset is {validation_dataset}\n")
print(f"Test Dataset is {test_dataset}\n")

Train Dataset is Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 120319
})

Validation Dataset is Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10000
})

Test Dataset is Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})



In [ ]:
def clean_text(text):
  text = re.sub('\s+', ' ', text)
  text = text.strip()
  return text

In [ ]:
def apply_clean_text(examples):
  cleaned_contexts = [clean_text(c) for c in examples['context']]
  cleaned_questions = [clean_text(q) for q in examples['question']]
  return {
      "context": cleaned_contexts,
      "question": cleaned_questions
  }


In [ ]:
train_dataset = train_dataset.map(
    apply_clean_text,
    batched = True,
    num_proc = 4
)
validation_dataset = validation_dataset.map(
    apply_clean_text,
    batched = True,
    num_proc = 4
)
test_dataset = test_dataset.map(
    apply_clean_text,
    batched = True,
    num_proc = 4
)

Map (num_proc=4):   0%|          | 0/120319 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def preprocess_training_examples(examples):
  tokenized = tokenizer(
      examples["question"],
      examples["context"],
      truncation = True,
      stride = 128,
      return_overflowing_tokens = True,
      return_offsets_mapping = True,
      padding = "max_length"
  )
  offset_mapping = tokenized["offset_mapping"]
  sample_map = tokenized.pop("overflow_to_sample_mapping")
  start_positions = []
  end_positions = []
  for i, offsets in enumerate(offset_mapping):
    sample_idx = sample_map[i]
    answers = examples["answers"][sample_idx]
    if len(answers["text"]) == 0:
      start_positions.append(0)
      end_positions.append(0)
      continue
    answer_text = answers["text"][0]
    answer_start_char = answers["answer_start"][0]
    answer_end_char = answer_start_char + len(answer_text)
    start_token_idx = 0
    end_token_idx = 0
    for idx, (start,end) in enumerate(offsets):
      if start <= answer_start_char < end:
        start_token_idx = idx
      if start < answer_end_char <= end:
        end_token_idx = idx
        break
    start_positions.append(start_token_idx)
    end_positions.append(end_token_idx)
  tokenized["start_positions"] = start_positions
  tokenized["end_positions"] = end_positions
  return tokenized

In [ ]:
tokenized_train = train_dataset.map(
    preprocess_training_examples,
    batched = True,
    remove_columns = train_dataset.column_names
)
tokenized_test = test_dataset.map(
    preprocess_training_examples,
    batched = True,
    remove_columns = test_dataset.column_names
)
tokenized_validation = validation_dataset.map(
    preprocess_training_examples,
    batched = True,
    remove_columns = validation_dataset.column_names
)

Map:   0%|          | 0/120319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenized_train.set_format("torch", columns = ["input_ids", "attention_mask", "start_positions", "end_positions"])
tokenized_test.set_format("torch", columns = ["input_ids", "attention_mask", "start_positions", "end_positions"])
tokenized_validation.set_format("torch", columns = ["input_ids", "attention_mask", "start_positions", "end_positions"])

In [ ]:
train_dataloader = DataLoader(tokenized_train, shuffle = True, batch_size = 8)
test_dataloader = DataLoader(tokenized_test, shuffle = True, batch_size = 8)
validation_dataloader = DataLoader(tokenized_validation, shuffle = True, batch_size = 8)

In [ ]:
sample_batch = next(iter(train_dataloader))
for k,v in sample_batch.items():
  print(k,v.shape)

input_ids torch.Size([8, 512])
attention_mask torch.Size([8, 512])
start_positions torch.Size([8])
end_positions torch.Size([8])


In [ ]:
processed_dataset = DatasetDict(
    {
        "train" : tokenized_train,
        "test" : tokenized_test,
        "validation" : tokenized_validation,
    }
)
processed_dataset.save_to_disk("processed_squad_v2")


Saving the dataset (0/4 shards):   0%|          | 0/120522 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11974 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10022 [00:00<?, ? examples/s]

In [ ]:
reload_processed_dataset = load_from_disk("processed_squad_v2")